# Initial setup of dependencies


### Author: Enrico Mendez
### Date: 03 April 2025
### Description of the Code:

This Jupyter Notebook demonstrates the process of training a YOLO (You Only Look Once) model for object detection using a custom dataset. This code was developped for Python 3.11.11 and `ultralytics 8.3.101` and `roboflow 1.1.60` libraries. Below is a summary of the workflow:

1. **Initial Setup**: 
    - Dependencies are installed and imported, including `ultralytics` for YOLO and `roboflow` for dataset management.
    - Google Drive is mounted to save and retrieve files.

2. **Dataset Import**:
    - The dataset is downloaded from Roboflow using the API key and project details.
    - The dataset is prepared for training by specifying the path to the `data.yaml` file.

3. **Model Training**:
    - A pre-trained YOLO model is loaded, and training is performed on the custom dataset.
    - Training parameters such as epochs, batch size, and image size are configured.

4. **Saving Results**:
    - Training results are copied to Google Drive for backup and further analysis.

5. **Validation**:
    - The trained model is validated using the dataset to evaluate its performance.
    - Validation results are also saved to Google Drive.

6. **Optional Deployment**:
    - The trained weights can be uploaded to Roboflow for deployment or further use.

This workflow is designed to streamline the process of training and validating a YOLO model for object detection tasks, leveraging the integration of Roboflow and Google Drive for efficient dataset management and result storage.


In [ ]:
# Run this cell if you are using Google Colab

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install the required packages

!pip install ultralytics
!pip install roboflow

In [ ]:
# Import the required libraries
from roboflow import Roboflow
from ultralytics import YOLO
import ultralytics
from datetime import datetime
import shutil

# Set the Roboflow API key
# Make sure to set your Roboflow API key in the environment variables
# If you are not using Google Colab, you can set it using the following command:
# Roboflow_api_key='YOUR_ROBOFLOW_API_KEY'
from google.colab import userdata         
Roboflow_api_key = userdata.get('Roboflow_api_key')


ultralytics.checks()

# Import dataset

In [ ]:
# Download the dataset from Roboflow
rf = Roboflow(api_key=Roboflow_api_key)
project = rf.workspace("enrico-mendez-research-s9yeb").project("capsicum-ir-box")
version = project.version(18)
dataset = version.download("yolov11")    

# Train

In [ ]:
dataset_path = dataset.location + "/data.yaml"
print(dataset_path)
model = YOLO("yolo11n.pt")

## If you want to continue training from a previous checkpoint, uncomment the following lines:
# model_path = "/content/drive/MyDrive/maturity_11n_v1/weights/last.pt"
# model = YOLO(model_path)

In [ ]:
# Train the model
results = model.train(data=dataset_path, epochs=500, batch = 0.9, cache='ram', imgsz = 1280, patience = 100)

In [ ]:
# Copy results to Google drive
import shutil
model_name = "yolo_11n"     # Model name
dataset_name = "2v1"         # Dataset name   

folder_name = f"{model_name}_{dataset_name}" 

results_path = '/content/runs/detect/train'
shutil.copytree(results_path, f'/content/drive/My Drive/{folder_name}')

# Validate model

In [ ]:
# Validate

model_path = "/content/v18n.pt"
model = YOLO(model_path)
dataset_path = dataset.location + "/data.yaml"
val = model.val(data = dataset_path)

In [ ]:
# Copy validation results to Google drive
nombre_carpeta = "validation"
results_path = '/content/runs/detect/val'
shutil.copytree(results_path, f'/content/drive/My Drive/{nombre_carpeta}')

# Upload wieghts

In [ ]:
# Upload the model weights to roboflow
rf = Roboflow(api_key=Roboflow_api_key)
project = rf.workspace("enrico-mendez-research-s9yeb").project("capsicum-ir-box")
version = project.version(18)
path = results.save_dir
weights_filename = "weights/best.pt"
version.deploy("yolo11",path, weights_filename)